In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv


load_dotenv()  # take environment variables

# How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html
DATABRICKS_TOKEN = os.environ.get('DATABRICKS_TOKEN')
# Alternatively in a Databricks notebook you can use this:
# DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url=f"{os.environ.get('DATABRICKS_HOST')}/serving-endpoints"
)

image_url = os.environ["IMAGE_SAMPLE_URL"]
image_format = os.environ["FILE_SUFFIX"]

from urllib.request import urlopen, Request
import base64

request = Request(image_url, headers={"User-Agent": "Mozilla/5.0"})
image_data = base64.b64encode(urlopen(request).read()).decode("utf-8")
data_url = f"data:image/{image_format};base64,{image_data}"

In [ ]:
import requests
import IPython.display as Disp

Disp.Image(requests.get(image_url).content)

In [ ]:
prompt = os.environ["PROMPT"].replace("\\n","\n")

In [ ]:
response = client.chat.completions.create(
    model="databricks-claude-3-7-sonnet",
    messages=[
        {
            "role": "user",
            "content": [ {
                "type": "text",
                "text": prompt
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": data_url
                }
            }]
        }
    ]
)

In [ ]:
print(f"{response.choices[0].message.role}: {response.choices[0].message.content}")
print("Model:", response.model)
print("Usage:")
print("\tPrompt tokens:", response.usage.prompt_tokens)
print("\tCompletion tokens:", response.usage.completion_tokens)
print("\tTotal tokens:", response.usage.total_tokens)